In [130]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [131]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


# **Preprocessing**

**Train**

**pooling**

**2nd covolution layer**

**3rd**

**flattening**

**connections**

**outputlayer**

# **Training**

# single prediction single photo

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import confusion_matrix, classification_report


In [4]:
train_dir = r'D:\Project\FaceDetection\Train'
test_dir  = r'D:\Project\FaceDetection\Test'


In [5]:
# Training data (with augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Test data (only normalization)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load datasets
training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    seed=42,
    classes=['WithMask', 'WithoutMask']
)

test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    seed=42,
    classes=['WithMask', 'WithoutMask']
)

print("\nClass mapping:", training_set.class_indices)


Found 10000 images belonging to 2 classes.
Found 992 images belonging to 2 classes.

Class mapping: {'WithMask': 0, 'WithoutMask': 1}


In [6]:
cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

cnn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 32)      896       
                                                                 
 batch_normalization_3 (Bat  (None, 128, 128, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 batch_normalization_4 (Bat  (None, 64, 64, 64)        256       
 chNormalization)                                                
                                                      

In [7]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn.fit(
    x=training_set,
    validation_data=test_set,
    epochs=20
)


Epoch 1/20
313/313 [==============================] - 82s 258ms/step - loss: 0.3867 - accuracy: 0.9506 - val_loss: 1.8593 - val_accuracy: 0.8024
Epoch 2/20
313/313 [==============================] - 120s 382ms/step - loss: 0.0815 - accuracy: 0.9797 - val_loss: 0.0692 - val_accuracy: 0.9849
Epoch 3/20
313/313 [==============================] - 86s 276ms/step - loss: 0.0831 - accuracy: 0.9794 - val_loss: 0.3928 - val_accuracy: 0.9466
Epoch 4/20
313/313 [==============================] - 124s 398ms/step - loss: 0.0722 - accuracy: 0.9820 - val_loss: 0.2374 - val_accuracy: 0.9647
Epoch 5/20
313/313 [==============================] - 130s 412ms/step - loss: 0.0728 - accuracy: 0.9828 - val_loss: 0.2104 - val_accuracy: 0.9516
Epoch 6/20
313/313 [==============================] - 169s 537ms/step - loss: 0.0734 - accuracy: 0.9811 - val_loss: 0.2558 - val_accuracy: 0.9466
Epoch 7/20
313/313 [==============================] - 75s 239ms/step - loss: 0.0408 - accuracy: 0.9879 - val_loss: 0.0614 - va

In [24]:
test_image_path = r'D:\Project\FaceDetection\WIN_20251031_19_13_44_Pro.png'

test_image = image.load_img(test_image_path, target_size=(128, 128))
test_image = image.img_to_array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image)

if result[0][0] > 0.5:
    prediction = "WithoutMask"
else:
    prediction = "WithMask"

print(f"\nPrediction for {test_image_path}: {prediction}")


1/1 [==============================] - 0s 29ms/step

Prediction for D:\Project\FaceDetection\WIN_20251031_19_13_44_Pro.png: WithMask


In [9]:
test_set.reset()
predictions = cnn.predict(test_set)
predicted_labels = (predictions > 0.5).astype(int).flatten()
true_labels = test_set.classes

print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, target_names=['WithMask', 'WithoutMask']))


31/31 [==============================] - 2s 58ms/step

Confusion Matrix:
[[244 239]
 [232 277]]

Classification Report:
              precision    recall  f1-score   support

    WithMask       0.51      0.51      0.51       483
 WithoutMask       0.54      0.54      0.54       509

    accuracy                           0.53       992
   macro avg       0.52      0.52      0.52       992
weighted avg       0.53      0.53      0.53       992



In [16]:
cnn.save("face_mask_cnn_model.h5")
print("\n✅ Model saved as face_mask_cnn_model.h5")



✅ Model saved as face_mask_cnn_model.h5


c:\Users\Mayur\miniconda3\envs\gymenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
